# **AI Opportunity Navigator — Capstone Project**
A multi-agent assistant that reads your background, finds relevant opportunities, and gives you a clear plan for what to do next.

---

## 🔹 1. Introduction

Career decisions are messy — too many options, not enough clarity.

While going through the Google × Kaggle AI Agents Intensive, I realized that the same multi-agent ideas we practiced can actually be used to solve this problem in real life.

So I built **AI Opportunity Navigator**, a personalized multi-agent system that looks at a student's or job seeker's background and suggests:

- what roles or opportunities match them  
- why they match  
- what steps they should take next  

The goal is simple: **reduce confusion and give direction.**

---

## 🔹 2. What Problem This Agent Solves

Many people (including me) run into:

- Too many choices and not enough clarity  
- Not knowing which roles match their skills  
- Confusion about which professors / labs / competitions fit their interests  
- Being unsure whether to apply, learn more, or switch direction  
- Generic career tools that don’t really adapt to their background  

I wanted something that actually **understands your profile and reacts to it**, instead of giving template advice.

---

## 🔹 3. High-Level Idea

The AI Opportunity Navigator:

- reads a short background from the user  
- extracts skills, interests, and goals  
- searches or generates matched opportunities  
- scores them  
- creates a simple action plan with “do this next” steps  

Under the hood, it uses multiple agents working together:

- **Profile Agent** – understands the user  
- **Opportunity Finder Agent** – proposes possible paths  
- **Scoring Agent** – ranks options  
- **Planner Agent** – turns everything into a clear next-step plan  
- **Orchestrator** – coordinates the workflow

Next, we’ll define the tools and agents.


## 🔹 4. Agent Architecture (How It Actually Works)

Instead of one giant “do everything” agent, I split the logic into small, focused agents:

1. **Profile Agent**  
   - Reads the user’s background: education, projects, skills, goals  
   - Extracts a clean JSON-style profile with:
     - `skills`
     - `interests`
     - `experience`
     - `goals`
   - This becomes the shared context for the other agents.

2. **Opportunity Finder Agent**  
   - Uses the profile to brainstorm possible directions:
     - roles (e.g., “ML Engineer intern”, “Cloud Support”)
     - learning paths (courses, topics)
     - projects or competitions
   - It doesn’t just throw random ideas — it tries to match skills and interests.

3. **Scoring Agent**  
   - Looks at each opportunity and scores it on:
     - skill match (how well it fits the current profile)
     - difficulty level
     - relevance to long-term goals  
   - Returns a short justification for each score.

4. **Planner Agent**  
   - Takes the top opportunities and converts them into a simple, clear plan:
     - what to focus on now
     - what to apply for
     - what to learn or build next
   - Output is a human-readable roadmap.

5. **Orchestrator (Multi-Agent Workflow)**  
   - Controls the overall flow:
     1. Run **Profile Agent** on the user’s background  
     2. Pass its output to **Opportunity Finder Agent**  
     3. Pass opportunities + profile to **Scoring Agent**  
     4. Pass best options to **Planner Agent**  
   - Implemented using a **sequential multi-agent workflow**, similar to the course notebooks.




## 🔧 Step 1 — Setup, Imports, and Authentication

This section initializes everything needed for the project:

- Imports Google ADK components  
- Imports Gemini function-calling tools  
- Sets up the session and runner  
- Loads the **Google API Key** securely using Kaggle Secrets  

If you're running this notebook on Kaggle:

1. Go to **Settings → Secrets**  
2. Create a new secret named **`GOOGLE_API_KEY`**  
3. Paste your Gemini API key  

Once this is done, the code below will automatically authenticate and prepare the environment.




In [1]:
# ===== Cell 1: Imports + auth =====
import os

from kaggle_secrets import UserSecretsClient
from google.genai import types
from google.genai.types import Part, UserContent

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.function_tool import FunctionTool

# Get GOOGLE_API_KEY from Kaggle secrets
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print("❌ ERROR: Add 'GOOGLE_API_KEY' to Kaggle Secrets first.", e)


✅ Setup and authentication complete.


In [2]:
# ===== Retry options for the model =====
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 502, 503],
)

print("✅ Retry config ready.")


✅ Retry config ready.


### 🔧 Step 2 — Define Core Tools
These Python functions extract the user profile, generate opportunity ideas, and score them.


In [3]:
# ===== Core Python tools =====

def extract_profile(profile_text: str) -> dict:
    """
    Take a free-form profile text and return a structured profile.
    Works well with text like:
    Background: ...
    Skills: ...
    Interests: ...
    Goals: ...
    """
    lines = profile_text.splitlines()
    profile = {
        "raw_text": profile_text,
        "background": "",
        "skills": [],
        "interests": [],
        "goals": [],
    }

    current_key = None
    for line in lines:
        line = line.strip()
        lower = line.lower()

        if lower.startswith("background:"):
            current_key = "background"
            profile["background"] = line.split(":", 1)[1].strip()
        elif lower.startswith("skills:"):
            current_key = "skills"
            skills_str = line.split(":", 1)[1].strip()
            profile["skills"] = [s.strip() for s in skills_str.split(",") if s.strip()]
        elif lower.startswith("interests:"):
            current_key = "interests"
            int_str = line.split(":", 1)[1].strip()
            profile["interests"] = [s.strip() for s in int_str.split(",") if s.strip()]
        elif lower.startswith("goals:"):
            current_key = "goals"
            goals_str = line.split(":", 1)[1].strip()
            profile["goals"] = [s.strip() for s in goals_str.split(",") if s.strip()]
        else:
            # continuation lines – append to the last section if exists
            if current_key == "background" and line:
                profile["background"] += " " + line

    return profile


def suggest_opportunities(profile: dict) -> dict:
    """
    Very simple rule-based suggestion engine.
    Returns a dict with profile + list of opportunity dicts.
    """
    skills = [s.lower() for s in profile.get("skills", [])]
    interests = [i.lower() for i in profile.get("interests", [])]
    goals = [g.lower() for g in profile.get("goals", [])]

    opps = []

    # Research assistant
    if "research" in " ".join(interests + goals):
        opps.append({
            "type": "research",
            "title": "Graduate Research Assistant — AI / ML",
            "reason": "You mentioned interest in research and AI/ML.",
        })

    # Cloud + AI internship
    if "cloud" in " ".join(skills + interests + goals):
        opps.append({
            "type": "internship",
            "title": "Cloud + AI Engineering Internship",
            "reason": "You have cloud + AI related skills and goals.",
        })

    # Agentic AI projects
    if "agent" in " ".join(interests + goals) or "agents" in " ".join(interests + goals):
        opps.append({
            "type": "project",
            "title": "Applied ML + Agentic AI Project",
            "reason": "You’re interested in building agentic AI systems.",
        })

    # Fallback
    if not opps:
        opps.append({
            "type": "learning",
            "title": "Structured 90-day learning plan",
            "reason": "No specific interests detected, so start with a broad ML/AI roadmap.",
        })

    return {
        "profile": profile,
        "opportunities": opps,
    }


def score_opportunities(profile_with_opps: dict) -> dict:
    """
    Add a simple 'match_score' to each opportunity and sort them.
    """
    profile = profile_with_opps.get("profile", {})
    skills = {s.lower() for s in profile.get("skills", [])}
    interests = {i.lower() for i in profile.get("interests", [])}
    goals = {g.lower() for g in profile.get("goals", [])}

    for opp in profile_with_opps.get("opportunities", []):
        text = (opp.get("title", "") + " " + opp.get("reason", "")).lower()
        score = 0

        # basic scoring
        for w in skills:
            if w and w in text:
                score += 2
        for w in interests:
            if w and w in text:
                score += 1
        for w in goals:
            if w and w in text:
                score += 2

        opp["match_score"] = score

    # sort by score desc
    profile_with_opps["opportunities"] = sorted(
        profile_with_opps.get("opportunities", []),
        key=lambda o: o.get("match_score", 0),
        reverse=True,
    )

    return profile_with_opps

print("✅ Python tools defined.")


✅ Python tools defined.


### 🧰 Step 4 — Convert Python Functions into ADK Tools
These tools allow the agents to call our Python functions during the workflow.


In [4]:

# ===== Cell 4: Wrap Python functions as ADK tools (simple version) =====

from google.adk.tools.function_tool import FunctionTool

profile_tool = FunctionTool(extract_profile)
opportunity_tool = FunctionTool(suggest_opportunities)
scoring_tool = FunctionTool(score_opportunities)

print("✅ Function tools created (simple ADK version).")



✅ Function tools created (simple ADK version).


### 🧠 Step 5 — Define the LLM Agents & Build the Orchestrator
This step sets up the core multi-agent workflow.  
Each agent has a clear responsibility:

- **ProfileAgent** → Understands the user background  
- **OpportunityAgent** → Suggests relevant roles, projects, and learning paths  
- **PlannerAgent** → Scores options and creates a structured 60–90 day plan  

These agents are then connected using a **SequentialAgent orchestration flow**, so the output of one becomes the input of the next.


In [5]:
# ===== Define LLM agents + Sequential orchestrator (no retry_options) =====

# ProfileAgent – understands the user background
profile_agent = LlmAgent(
    name="profile_agent",
    model="gemini-2.5-flash-lite",
    tools=[profile_tool],
    instruction=(
        "You are ProfileAgent. Read the user's background, skills, "
        "interests, and goals from free-form text. "
        "Call the extract_profile tool to structure it into a profile "
        "dictionary. Then briefly summarize who this person is."
    ),
)

# OpportunityAgent – suggests options
opportunity_agent = LlmAgent(
    name="opportunity_agent",
    model="gemini-2.5-flash-lite",
    tools=[opportunity_tool],
    instruction=(
        "You are OpportunityAgent. Given a structured profile, "
        "call suggest_opportunities to propose research roles, internships, "
        "projects, and learning paths that fit this person. "
        'Explain in 2–3 sentences why they match. '
        "Always keep the tone practical and encouraging."
    ),
)

# PlannerAgent – turns options into a concrete plan
planner_agent = LlmAgent(
    name="planner_agent",
    model="gemini-2.5-flash-lite",
    tools=[scoring_tool],
    instruction=(
        "You are PlannerAgent. Take the profile plus list of opportunities, "
        "call score_opportunities, then create a clear 60–90 day action plan. "
        "First list the top 3 opportunities with reasons. Then give a weekly "
        "plan grouped by Month 1 / Month 2 / Month 3."
    ),
)

# Orchestrator – runs everything in order
orchestrator = SequentialAgent(
    name="ai_opportunity_navigator",
    sub_agents=[profile_agent, opportunity_agent, planner_agent],
    description=(
        "Runs profile understanding, opportunity suggestion, and 60–90 day "
        "planning in sequence to guide the user on research, internships, "
        "and projects."
    ),
)

print("✅ Agents and orchestrator created.")


✅ Agents and orchestrator created.


### ⚙️ Step 6 — Create the Runner & Session
Now that all agents and the orchestrator are ready, we initialize the **InMemoryRunner**, which is responsible for executing the multi-agent pipeline.

We also create a **session**, which allows the agents to maintain context across multiple turns.  
This is important because our system will support follow-up questions and multi-step interaction.


In [6]:
# ===== Create InMemoryRunner + session =====

runner = InMemoryRunner(agent=orchestrator)

async def _create_session():
    return await runner.session_service.create_session(
        app_name=runner.app_name,
        user_id="demo_user",
    )

session = await _create_session()  # works in Kaggle notebooks
print("✅ Runner and session ready.")
print("Session ID:", session.id)


✅ Runner and session ready.
Session ID: 933b1230-c58c-4bbd-8f47-8de13e6022c8


### 🧪 Step 7 — Test the Multi-Agent Pipeline
Before integrating everything into a UI, we test the entire workflow by sending a sample profile into the system.

This runs the full sequence:
1️⃣ ProfileAgent → extracts structured profile  
2️⃣ OpportunityAgent → suggests relevant roles & paths  
3️⃣ PlannerAgent → generates a 60–90 day plan  

The output printed below confirms that all agents are working together correctly.


In [7]:
# ===== Test the multi-agent pipeline =====

test_profile_text = """
Background: MS in Computer Science at Wright State University, focusing on AI, cloud, and distributed systems.
Skills: Python, machine learning, deep learning, PyTorch, cloud computing, networking, Git.
Interests: AI agents, research assistant roles, Kaggle competitions, cloud + AI projects.
Goals: Build strong agentic AI projects, get a research/internship role, and move towards AI/ML engineering.
"""

content = UserContent(parts=[Part(text=test_profile_text)])

print("### Running AI Opportunity Navigator on your profile...\n")

final_text = ""


for event in runner.run(
    user_id=session.user_id,
    session_id=session.id,
    new_message=content,
):
    if getattr(event, "content", None):
        for p in event.content.parts or []:
            if getattr(p, "text", None):
                print(p.text, "\n")
                final_text += p.text + "\n\n"

print("\n✅ Done.")


### Running AI Opportunity Navigator on your profile...



This individual has a Master's degree in Computer Science with a specialization in AI, cloud, and distributed systems. They are proficient in Python, machine learning, deep learning, PyTorch, cloud computing, networking, and Git. Their interests lie in AI agents, research, Kaggle competitions, and cloud/AI projects. Their goals are to develop robust AI agent projects, secure a research or internship position, and advance into an AI/ML engineering role. 



I've found a few opportunities that seem like a great fit for your background and goals. A Graduate Research Assistant role focusing on AI/ML would align well with your interest in research and your advanced studies. Additionally, a Cloud + AI Engineering Internship would leverage your existing cloud computing skills and your goal of working with AI. Finally, an Applied ML + Agentic AI Project would be a perfect way to build on your interest in AI agents and develop strong project experience. 



Here are the top 3 opportunities based on your profile:

1.  **Graduate Research Assistant — AI / ML**: This role aligns perfectly with your interest in research and your advanced studies in AI/ML.
2.  **Cloud + AI Engineering Internship**: This internship will allow you to leverage your existing skills in cloud computing and your goal of working with AI technologies.
3.  **Applied ML + Agentic AI Project**: This project is an excellent opportunity to deepen your experience with AI agents and build a strong portfolio of agentic AI systems.

Here is a 60-90 day action plan:

**Month 1: Foundation and Exploration**

*   **Week 1-2**:
    *   **Research Assistant**: Identify potential PIs and labs at Wright State University or other institutions whose research aligns with your AI/ML interests. Review their publications and ongoing projects.
    *   **Internship**: Begin researching companies that offer Cloud + AI internships. Update your resume and LinkedIn profile to highlight relevant s

### 🧱 Step 8 — UI Helper (Convert Form Inputs into Profile Text)

To prepare the user data for the multi-agent pipeline, we convert all the UI form inputs  
(name, background, skills, interests, goals, etc.) into a structured text block.

This formatted profile text is what gets passed into the SequentialAgent workflow.


In [8]:
# ===== UI Helper: build profile text from form inputs =====

from google.genai.types import Part, UserContent
import gradio as gr

def build_profile_text_ui(
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
):
    name = (name or "").strip() or "User"
    background = (background or "").strip()
    skills = (skills or "").strip()
    interests = (interests or "").strip()
    goals = (goals or "").strip()

    grad_status = "Yes" if graduated else "No"
    role_types = ", ".join(role_types or []) if role_types else "Not specified"

    lines = [
        f"Name: {name}",
        f"Graduation completed: {grad_status}",
        f"Experience level: {exp_level or 'Not specified'}",
        f"Target roles: {role_types}",
        "",
        f"Background: {background or 'Not provided'}",
        f"Skills: {skills or 'Not provided'}",
        f"Interests: {interests or 'Not provided'}",
        f"Goals: {goals or 'Not provided'}",
    ]
    return "\n".join(lines)


### 🧠 Step 9 — Core Logic (Run the Navigator)

This function takes the user’s form inputs, formats them into a profile,  
builds the full prompt, and then passes everything into the multi-agent pipeline.  
The final consolidated output (summary, opportunities, and 60–90 day plan)  
is returned to the UI.


In [9]:
# ===== Core function: run AI Opportunity Navigator from UI inputs =====

def run_navigator_ui(
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
    extra_question,
):
    # Build profile text from the form
    profile_text = build_profile_text_ui(
        name,
        background,
        skills,
        interests,
        goals,
        graduated,
        role_types,
        exp_level,
    )

    # If user doesn't type a question, use a default one
    user_message = extra_question.strip() or "Give me the best opportunities and a 60–90 day plan."

    full_prompt = (
        "You are AI Opportunity Navigator. "
        "Use the profile below and the user's message to give practical, "
        "specific suggestions (research roles, internships, projects, and a 60–90 day plan).\n\n"
        "=== USER PROFILE ===\n"
        f"{profile_text}\n\n"
        "=== USER MESSAGE ===\n"
        f"{user_message}\n"
    )

    content = UserContent(parts=[Part(text=full_prompt)])

    # Call your existing multi-agent pipeline
    response_text = ""
    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content,
    ):
        if getattr(event, "content", None):
            for p in event.content.parts or []:
                if getattr(p, "text", None):
                    response_text += p.text + "\n\n"

    response_text = response_text.strip() or "I couldn't generate a response this time."
    return response_text


### 🖥️ Step 11 — Build the Interactive UI (Profile + Chat)

This is the final layer of the project — a clean Gradio interface that feels like a real product.

On the left, the user fills in their background, skills, interests, goals, and experience level.  
On the right, they interact with a chat-like interface:

- **Start** → generates the first personalized summary, matching opportunities, and 60–90 day plan  
- **Follow-up chat** → the agent remembers the conversation, asks questions if needed, and adjusts the plan  
- **Clear chat** → resets the conversation  

This UI brings the whole multi-agent workflow to life.


In [10]:
# ===== UI: Profile Form + Chat with Start button =====

import gradio as gr
from google.genai.types import Part, UserContent

# Helper: build profile text from form inputs
def build_profile_text_ui(
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
):
    name = (name or "").strip() or "User"
    background = (background or "").strip()
    skills = (skills or "").strip()
    interests = (interests or "").strip()
    goals = (goals or "").strip()

    grad_status = "Yes" if graduated else "No"
    role_types = ", ".join(role_types or []) if role_types else "Not specified"

    lines = [
        f"Name: {name}",
        f"Graduation completed: {grad_status}",
        f"Experience level: {exp_level or 'Not specified'}",
        f"Target roles: {role_types}",
        "",
        f"Background: {background or 'Not provided'}",
        f"Skills: {skills or 'Not provided'}",
        f"Interests: {interests or 'Not provided'}",
        f"Goals: {goals or 'Not provided'}",
    ]
    return "\n".join(lines)


# Core engine: shared by Start + follow-up chat
def run_navigator_core(
    user_message,
    chat_history,
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
    is_start=False,
):
    if chat_history is None:
        chat_history = []

    # Build profile context from the form
    profile_text = build_profile_text_ui(
        name,
        background,
        skills,
        interests,
        goals,
        graduated,
        role_types,
        exp_level,
    )

    # Turn previous chat history into text
    history_text = ""
    for u, a in chat_history:
        history_text += f"User: {u}\nAssistant: {a}\n\n"

    # If this is the first Start click, we auto-define the user message
    if is_start:
        user_message = (
            "Use my profile and generate a personalized summary, "
            "top matching opportunities (RA, internships, projects), "
            "and a clear 60–90 day plan. "
            "If you need more information about me, ask 1–3 specific questions."
        )
    else:
        user_message = (user_message or "").strip()
        if not user_message:
            # no new question, just return old history
            return chat_history

    # What we send to the orchestrator
    full_prompt = (
        "You are AI Opportunity Navigator. "
        "You know the user's background and goals from the profile below. "
        "Use that profile and the conversation so far to give practical, "
        "specific suggestions (research roles, internships, projects, and 60–90 day plans). "
        "If you are missing important information, ask direct follow-up questions.\n\n"
        "=== USER PROFILE ===\n"
        f"{profile_text}\n\n"
        "=== CONVERSATION SO FAR ===\n"
        f"{history_text}\n"
        "=== LATEST USER MESSAGE ===\n"
        f"{user_message}\n"
    )

    content = UserContent(parts=[Part(text=full_prompt)])

    # Run the multi-agent pipeline
    assistant_reply = ""
    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content,
    ):
        if getattr(event, "content", None):
            for p in event.content.parts or []:
                if getattr(p, "text", None):
                    assistant_reply += p.text + "\n\n"

    assistant_reply = assistant_reply.strip() or "I couldn't generate a response this time."

    # Add to chat history (user bubble + assistant bubble)
    chat_history = chat_history + [(user_message, assistant_reply)]
    return chat_history


# Wrapper for Start button
def start_conversation(
    chat_history,
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
):
    return run_navigator_core(
        user_message="",  # will be auto-filled inside
        chat_history=chat_history,
        name=name,
        background=background,
        skills=skills,
        interests=interests,
        goals=goals,
        graduated=graduated,
        role_types=role_types,
        exp_level=exp_level,
        is_start=True,
    )


# Wrapper for follow-up messages
def continue_conversation(
    message,
    chat_history,
    name,
    background,
    skills,
    interests,
    goals,
    graduated,
    role_types,
    exp_level,
):
    return run_navigator_core(
        user_message=message,
        chat_history=chat_history,
        name=name,
        background=background,
        skills=skills,
        interests=interests,
        goals=goals,
        graduated=graduated,
        role_types=role_types,
        exp_level=exp_level,
        is_start=False,
    )


# ===== Build Gradio UI =====

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 🎯 AI Opportunity Navigator

        1. Fill in your profile on the left (once).  
        2. Click **Start** to generate your first personalized plan.  
        3. Then ask follow-up questions on the right to refine it.  
        """
    )

    with gr.Row():
        # -------- LEFT: Profile Form --------
        with gr.Column(scale=1):
            gr.Markdown("### 1️⃣ Your Profile")

            name = gr.Textbox(
                label="Name",
                value="Rishindra",
                placeholder="Your name",
            )

            background = gr.Textbox(
                label="Background",
                placeholder="e.g., MS in CS at Wright State University, focus on AI, cloud, distributed systems…",
                lines=3,
            )

            skills = gr.Textbox(
                label="Skills",
                placeholder="e.g., Python, ML, DL, PyTorch, Cloud, Networking, Git…",
                lines=3,
            )

            interests = gr.Textbox(
                label="Interests",
                placeholder="e.g., AI agents, research assistant roles, Kaggle, cloud+AI projects…",
                lines=3,
            )

            goals = gr.Textbox(
                label="Goals",
                placeholder="e.g., Build strong agentic AI projects, get RA/internship, move towards AI/ML engineer…",
                lines=3,
            )

            graduated = gr.Checkbox(
                label="Graduation completed?",
                value=False,
            )

            role_types = gr.CheckboxGroup(
                label="What are you mainly looking for?",
                choices=[
                    "Research assistant (RA)",
                    "Internship",
                    "Entry-level job",
                    "Mid-level role",
                    "Just exploring options",
                ],
                value=["Research assistant (RA)", "Internship"],
            )

            exp_level = gr.Radio(
                label="Experience level",
                choices=[
                    "Student / Fresher",
                    "0–1 years industry",
                    "1–3 years industry",
                    "3+ years industry",
                ],
                value="Student / Fresher",
            )

        # -------- RIGHT: Chat Interface --------
        with gr.Column(scale=2):
            gr.Markdown("### 2️⃣ Chat with the AI Opportunity Navigator")

            chatbot = gr.Chatbot(label="AI Opportunity Navigator")

            # Start button at top of chat
            start_btn = gr.Button("🚀 Start with my profile")

            msg = gr.Textbox(
                label="Your follow-up message",
                placeholder="Ask anything like: 'Focus more on RA roles' or 'Adjust the plan for 30 days only.'",
            )

            send_btn = gr.Button("Send")
            clear_btn = gr.Button("Clear chat")

            # Wire Start button
            start_btn.click(
                fn=start_conversation,
                inputs=[
                    chatbot,      # existing chat history
                    name,
                    background,
                    skills,
                    interests,
                    goals,
                    graduated,
                    role_types,
                    exp_level,
                ],
                outputs=chatbot,
            )

            # Wire Send (follow-up messages)
            send_btn.click(
                fn=continue_conversation,
                inputs=[
                    msg,
                    chatbot,
                    name,
                    background,
                    skills,
                    interests,
                    goals,
                    graduated,
                    role_types,
                    exp_level,
                ],
                outputs=chatbot,
            ).then(
                fn=lambda: "",
                inputs=None,
                outputs=msg,  # clear message box
            )

            clear_btn.click(
                fn=lambda: [],
                inputs=None,
                outputs=chatbot,
            )

    demo.launch()


/tmp/ipykernel_13/1182187368.py:259: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="AI Opportunity Navigator")


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://f36cfad26ae59bcae3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Copyright 2025 Google LLC.

In [11]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.